In [2]:
import pandas as pd
import numpy as np
import plotly.figure_factory as ff
from sklearn.metrics import confusion_matrix
import plotly.express as px
import os, librosa
from tqdm import tqdm
import torchaudio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import IPython.display as ipd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from keras.utils import to_categorical
from keras import layers, Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers
import tensorflow as tf


In [13]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())



[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10752792452178624065
xla_global_id: -1
]


In [3]:


# Replace 'filename.csv' with the path to your CSV file
df = pd.read_csv('extracted_features.csv')

# Print the DataFrame
df

,zcr_mean,energy_mean,energy_entropy_mean,spectral_centroid_mean,spectral_spread_mean,spectral_entropy_mean,spectral_flux_mean,spectral_rolloff_mean,mfcc_1_mean,mfcc_2_mean,...,delta chroma_7_std,delta chroma_8_std,delta chroma_9_std,delta chroma_10_std,delta chroma_11_std,delta chroma_12_std,delta chroma_std_std,bpm,ratio,Classes
0,0.059571,0.013810,2.760099,0.153608,0.208083,0.319367,0.007084,0.073787,-28.106687,2.597397,...,0.031851,0.002397,0.016663,0.012523,0.017773,0.021164,0.016804,150.000000,0.333333,children_playing
1,0.068853,0.014112,3.098093,0.140553,0.169558,0.410356,0.005330,0.093027,-24.657740,0.601445,...,0.010810,0.005635,0.009398,0.005186,0.004059,0.004634,0.005723,85.714286,0.175000,children_playing
2,0.091936,0.038419,3.210188,0.150212,0.186407,0.992321,0.002087,0.120240,-22.550267,-0.042209,...,0.024040,0.004511,0.003712,0.007746,0.011042,0.001289,0.007112,75.000000,0.161290,children_playing
3,0.059038,0.040164,3.264842,0.106313,0.135973,0.436038,0.001945,0.085057,-20.523216,1.338314,...,0.010670,0.005959,0.005821,0.003573,0.006179,0.004083,0.003294,42.857143,0.133333,children_playing
4,0.015945,0.041795,2.936177,0.087112,0.164892,0.049966,0.011352,0.014921,-24.693257,2.411662,...,0.181156,0.002499,0.004122,0.006625,0.023765,0.008123,0.038304,100.000000,0.179487,children_playing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5337,0.147925,0.017386,2.770395,0.212368,0.197951,0.762406,0.004987,0.162449,-27.419145,0.153562,...,0.007328,0.003617,0.002804,0.003573,0.001756,0.002327,0.001542,600.000000,0.000000,dog_bark
5338,0.020674,0.011507,2.094007,0.097402,0.132085,0.213117,0.102750,0.041304,-44.146212,1.485591,...,0.005292,0.004837,0.005457,0.008903,0.012438,0.017566,0.010079,75.000000,0.333333,dog_bark
5339,0.059832,0.021739,2.960517,0.135032,0.200646,0.495383,0.003148,0.072880,-22.306442,1.902591,...,0.012886,0.007575,0.004371,0.007339,0.008028,0.005705,0.002457,200.000000,1.000000,dog_bark
5340,0.069025,0.028611,2.586032,0.117883,0.131261,0.612880,0.001446,0.102041,-25.519868,-2.746113,...,0.000476,0.001529,0.000647,0.000310,0.003288,0.003033,0.000554,600.000000,0.000000,dog_bark


In [4]:
# Convert features into numpy array
x = df.drop(columns=['Classes']).values

# Encode classes
encoder = LabelEncoder()
y = encoder.fit_transform(df['Classes'])
y = to_categorical(y, num_classes=10)



In [8]:
x.shape

(5342, 138)

In [5]:
# Train-validation split
trainX, testX, trainY, testY = train_test_split(x, y, stratify=y, random_state=0)

In [11]:


model = Sequential([
    layers.Dense(512, activation='relu', input_shape=(138,)),
    layers.BatchNormalization(axis=-1),
    layers.Dropout(0.2),
    
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(axis=-1),
    layers.Dropout(0.2),
    
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(axis=-1),
    layers.Dropout(0.2),
    
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(axis=-1),
    layers.Dropout(0.2),
    
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(axis=-1),
    layers.Dropout(0.2),
    
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(axis=-1),
    layers.Dropout(0.2),
    
    layers.Dense(10, activation='softmax')
])


In [15]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=5e-4,
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=3,
    min_lr=1e-5
)

In [17]:
# Train the model
Train_model = model.fit(
    trainX, trainY,
    validation_data=(testX, testY),
    epochs=100,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.1605 - loss: 2.6871 - val_accuracy: 0.2680 - val_loss: 2.0818 - learning_rate: 0.0010
Epoch 2/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3046 - loss: 2.0225 - val_accuracy: 0.2927 - val_loss: 1.8743 - learning_rate: 0.0010
Epoch 3/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3803 - loss: 1.7148 - val_accuracy: 0.3503 - val_loss: 1.7367 - learning_rate: 0.0010
Epoch 4/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4659 - loss: 1.5016 - val_accuracy: 0.3765 - val_loss: 1.7066 - learning_rate: 0.0010
Epoch 5/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4807 - loss: 1.4299 - val_accuracy: 0.4558 - val_loss: 1.3984 - learning_rate: 0.0010
Epoch 6/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5092 - loss: 1.3628 - val_accuracy: 0.4633 - val_loss: 1.4817 - learning_rate: 0.0010
Epoch 7/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5452 - loss: 1.

In [19]:
# Compute confusion matrix
predicted = np.argmax(model.predict(testX), axis=1)
true = np.argmax(testY, axis=1)
matrix = confusion_matrix(true, predicted)

# Plot confusion matrix
labels = list(encoder.classes_)

fig = ff.create_annotated_heatmap(z=matrix, x=labels, y=labels, colorscale='Viridis')
fig.update_layout(
                  xaxis=dict(title='Predicted Class'),
                  yaxis=dict(title='True Class'),
                  font=dict(size=12),
                  width=1000,
                  height=500)
fig.show()

42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
